In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy.optimize import curve_fit
import scipy.signal as spy
import glob
import time
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
import imageio
import moviepy.editor as mpy
import pandas as pd
print(time.ctime())
%matplotlib inline
#%matplotlib notebook
#import cmocean
#import mpld3
#mpld3.enable_notebook()
plt.rcParams["font.family"] = "Times New Roman"

Fri Sep 21 11:04:44 2018


In [2]:
def data_gather(datasource, centered):
    #The function decides which data file you want based on the source, and whether 
    # you want the centered version or not. 
    #the offset is currently hardcoded based on a value all the values shared
    #still thinking about a way have it be variable
    if datasource == 0:
        directory = '/home/andrew/Documents/BNL 2018/Data/'
        file = directory + 'PolA000PolB000_DiodeCurrent_40,2mAW0028_H11-180711-162519-1.csv'
        data=np.loadtxt(file, dtype= float, delimiter=",", usecols = (0,1,2,3))

        if centered  ==  1:
            file = directory + 'PolA000PolB000_DiodeCurrent_35,2mAW0028_H11-180711-164524-1_cent.csv'
            data=np.loadtxt(file, dtype= float, delimiter=",", usecols = (0,1,2,3))
        x = data[:, 0]                                                          # places data in separate arrays
        y = data[:, 1]
        t = data[:, 2]
        a = data[:, 3]
    return x,y,t,a

In [6]:
def clean(x,y,t,a, n = np.empty(1), size = 500000000):
   # a function that for lack of a better term, cleans up the data. 
    #It applies the offset to time and converts to nanoseconds
    # Takes a much smaller slice of data, and has the option to delete old massive arrays
    xdata = x[0:size]
    ydata= y[0:size]
    adata = a[0:size]
    ndata = n[0:size]
    trough = t[0:size]
    trough = trough[0:size] -trough[0]
    tdata = trough/(2**12)*25
    if len(ndata) > 10:                  
        return (xdata,ydata,tdata,adata, ndata)
    else:
        return(xdata,ydata,tdata,adata)

In [4]:
x,y,t,a = data_gather(0,0)


In [7]:
xdata, ydata,tdata,adata = clean(x,y,t,a)

MemoryError: 

In [8]:
map = pd.DataFrame({'x': xdata,'y': ydata,'t': tdata,'a': adata})

NameError: name 'xdata' is not defined

In [ ]:
TOT = np.zeros([256,256] )
TOTcts = np.zeros([256,256])
#for i in range(1738):
for i in range(99999999):
    x_i = np.abs(xdata[i].astype(int)-255)
    y_i = ydata[i].astype(int) 
    TOT[x_i,y_i] += tdata[i]
    TOTcts[x_i,y_i]+= 1

In [ ]:
size =6
TOTblock = np.zeros([size,size])
TOTblockcts = np.zeros([size,size])
TOTblock1 = np.zeros([size,size])
TOTblockcts1 = np.zeros([size,size])
TOTblock2 = np.zeros([size,size])
TOTblockcts2 = np.zeros([size,size])
TOTblock3 = np.zeros([size,size])
TOTblockcts3 = np.zeros([size,size])
TOTblock4 = np.zeros([size,size])
TOTblockcts4 = np.zeros([size,size])
lowx = 114
lowy = 138
for i in range(752,775):
    if xdata[i] > lowx and xdata[i] < lowx + size and ydata[i] > lowy and ydata[i] < lowy + size:
        x_i = xdata[i].astype(int)-lowx
        y_i = ydata[i].astype(int)-lowy
        TOTblock[x_i,y_i] += adata[i]
        TOTblockcts[x_i,y_i]+= 1
lowx = 125
lowy = 187
for i in range(925,935):
    if xdata[i] > lowx and xdata[i] < lowx + size and ydata[i] > lowy and ydata[i] < lowy + size:
        x_i = xdata[i].astype(int)-lowx
        y_i = ydata[i].astype(int)-lowy
        TOTblock1[x_i,y_i] += adata[i]
        TOTblockcts1[x_i,y_i]+= 1
lowx = 180
lowy = 73        
for i in range(410,425):
    if xdata[i] > lowx and xdata[i] < lowx + size and ydata[i] > lowy and ydata[i] < lowy + size:
        x_i = xdata[i].astype(int)-lowx
        y_i = ydata[i].astype(int)-lowy
        TOTblock2[x_i,y_i] += adata[i]
        TOTblockcts2[x_i,y_i]+= 1
lowx = 163
lowy = 172 
for i in range(825,835):
    if xdata[i] > lowx and xdata[i] < lowx + size and ydata[i] > lowy and ydata[i] < lowy + size:
        x_i = xdata[i].astype(int)-lowx
        y_i = ydata[i].astype(int)-lowy
        TOTblock3[x_i,y_i] += adata[i]
        TOTblockcts3[x_i,y_i]+= 1
lowx = 144
lowy = 146
for i in range(371,380):
    if xdata[i] > lowx and xdata[i] < lowx + size and ydata[i] > lowy and ydata[i] < lowy + size:
        x_i = xdata[i].astype(int)-lowx
        y_i = ydata[i].astype(int)-lowy
        TOTblock4[x_i,y_i] += adata[i]
        TOTblockcts4[x_i,y_i]+= 1
        
TOTblockplot = TOTblock/TOTblockcts
TOTblockplot1 = TOTblock1/TOTblockcts1
TOTblockplot2 = TOTblock2/TOTblockcts2
TOTblockplot3 = TOTblock3/TOTblockcts3
TOTblockplot4 = TOTblock4/TOTblockcts4

In [ ]:
size =6
TOAblock = np.zeros([size,size])
TOAblockcts = np.zeros([size,size])
TOAblock1 = np.zeros([size,size])
TOAblockcts1 = np.zeros([size,size])
TOAblock2 = np.zeros([size,size])
TOAblockcts2 = np.zeros([size,size])
TOAblock3 = np.zeros([size,size])
TOAblockcts3 = np.zeros([size,size])
TOAblock4 = np.zeros([size,size])
TOAblockcts4 = np.zeros([size,size])
lowx = 114
lowy = 138
for i in range(756,762):
    if xdata[i] > lowx and xdata[i] < lowx + size and ydata[i] > lowy and ydata[i] < lowy + size:
        x_i = xdata[i].astype(int)-lowx
        y_i = ydata[i].astype(int)-lowy
        TOAblock[x_i,y_i] += (tdata[i]-tdata[756])
        TOAblockcts[x_i,y_i]+= 1
lowx = 125
lowy = 187
for i in range(925,929):
    if xdata[i] > lowx and xdata[i] < lowx + size and ydata[i] > lowy and ydata[i] < lowy + size:
        x_i = xdata[i].astype(int)-lowx
        y_i = ydata[i].astype(int)-lowy
        TOAblock1[x_i,y_i] += (tdata[i]-tdata[925])
        TOAblockcts1[x_i,y_i]+= 1
lowx = 180
lowy = 73        
for i in range(415,420):
    if xdata[i] > lowx and xdata[i] < lowx + size and ydata[i] > lowy and ydata[i] < lowy + size:
        x_i = xdata[i].astype(int)-lowx
        y_i = ydata[i].astype(int)-lowy
        TOAblock2[x_i,y_i] += (tdata[i]-tdata[415])
        TOAblockcts2[x_i,y_i]+= 1
lowx = 163
lowy = 172 
for i in range(830,835):
    if xdata[i] > lowx and xdata[i] < lowx + size and ydata[i] > lowy and ydata[i] < lowy + size:
        x_i = xdata[i].astype(int)-lowx
        y_i = ydata[i].astype(int)-lowy
        TOAblock3[x_i,y_i] += (tdata[i]-tdata[830])
        TOAblockcts3[x_i,y_i]+= 1
lowx = 144
lowy = 146
for i in range(371,380):
    if xdata[i] > lowx and xdata[i] < lowx + size and ydata[i] > lowy and ydata[i] < lowy + size:
        x_i = xdata[i].astype(int)-lowx
        y_i = ydata[i].astype(int)-lowy
        TOAblock4[x_i,y_i] += (tdata[i]-tdata[371])
        TOAblockcts4[x_i,y_i]+= 1
        
TOAblockplot = TOAblock/TOAblockcts
TOAblockplot1 = TOAblock1/TOAblockcts1
TOAblockplot2 = TOAblock2/TOAblockcts2
TOAblockplot3 = TOAblock3/TOAblockcts3
TOAblockplot4 = TOAblock4/TOAblockcts4

In [ ]:
blockplot = TOTblockcts
plt.figure(figsize = (12,12))
sns.heatmap(TOAblockplot,linewidths=.01,linecolor='black',cmap = 'plasma')
plt.show()

In [ ]:
plot = TOT/TOTcts
#plot = TOTcts
fig = plt.figure(figsize = (10,8))
ax0 = fig.add_axes([0, 0.2, 1, 0.8])
sns.heatmap(plot, ax = ax0,cmap = 'Spectral_r', yticklabels=False,xticklabels=False)
ax1 = fig.add_axes([0.06, 0.01, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(blockplot,ax=ax1, cmap = 'Spectral_r', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax1.spines[axis].set_visible(True)
    ax1.spines[axis].set_color('black')
    ax1.spines[axis].set_visible(True)
    ax1.spines[axis].set_color('black') 
ax2 = fig.add_axes([0.28, 0.01, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(blockplot1,ax=ax2,cmap = 'Spectral_r', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax2.spines[axis].set_visible(True)
    ax2.spines[axis].set_color('black')
    ax2.spines[axis].set_visible(True)
    ax2.spines[axis].set_color('black')
ax3 = fig.add_axes([0.5, 0.01, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(blockplot2,ax=ax3,  cmap = 'Spectral_r',yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax3.spines[axis].set_visible(True)
    ax3.spines[axis].set_color('black')
    ax3.spines[axis].set_visible(True)
    ax3.spines[axis].set_color('black') 
ax4 = fig.add_axes([0.72, 0.01, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(blockplot3,ax=ax4, cmap = 'Spectral_r', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax4.spines[axis].set_visible(True)
    ax4.spines[axis].set_color('black')
    ax4.spines[axis].set_visible(True)
    ax4.spines[axis].set_color('black') 
plt.show()

In [ ]:
TOT1 = np.zeros([256,256] )
TOTcts1 = np.zeros([256,256])
for i in range(1738):
#for i in range(999999999):
    x_i = np.abs(xdata[i].astype(int)-255)
    y_i = ydata[i].astype(int) 
    TOT1[x_i,y_i] += tdata[i]
    TOTcts1[x_i,y_i]+= 1

In [ ]:
import matplotlib.patches as ptc
plot = TOT/TOTcts
#plot = np.log(TOTcts + 1)
fig = plt.figure(figsize = (10,8))
ax = sns.heatmap(plot,cmap = 'Spectral_r', xticklabels = 20, yticklabels = 20)#, yticklabels=False,xticklabels=False)
'''
ax.axhline(y=56,xmin = 0.139, xmax = 0.26, color='k',linewidth=1)
ax.axhline(y=66,xmin = 0.139, xmax = 0.26, color='k',linewidth=1)
ax.axhline(y=76,xmin = 0.139, xmax = 0.26, color='k',linewidth=1)
ax.axhline(y=86,xmin = 0.139, xmax = 0.26, color='k',linewidth=1)
ax.axvline(x=36,ymin = 0.665, ymax = 0.78, color='k',linewidth=1)
ax.axvline(x=46,ymin = 0.665, ymax = 0.78, color='k',linewidth=1)
ax.axvline(x=56,ymin = 0.665, ymax = 0.78, color='k',linewidth=1)
ax.axvline(x=66,ymin = 0.665, ymax = 0.78, color='k',linewidth=1)
ax.axhline(y=100,xmin = 0.655, xmax = 0.81, color='k',linewidth=1)
ax.axhline(y=113,xmin = 0.655, xmax = 0.81, color='k',linewidth=1)
ax.axhline(y=126.5,xmin = 0.655, xmax = 0.81, color='k',linewidth=1)
ax.axhline(y=140,xmin = 0.655, xmax = 0.81, color='k',linewidth=1)
ax.axvline(x=168,ymin = 0.45, ymax = 0.61, color='k',linewidth=1)
ax.axvline(x=181.3,ymin = 0.45, ymax = 0.61, color='k',linewidth=1)
ax.axvline(x=195,ymin = 0.45, ymax = 0.61, color='k',linewidth=1)
ax.axvline(x=208,ymin = 0.45, ymax = 0.61, color='k',linewidth=1)
'''
plt.xlabel('X [px]')
plt.ylabel('Y [px]')
plt.savefig('TOA.eps', bbox_inches='tight')


In [ ]:
fig = plt.figure(figsize = (10,8))
ax1 = fig.add_axes([0.06, 0.01, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
cbar_ax = fig.add_axes([.3, .1, .03, .6])
cbar_ax2 = fig.add_axes([.7, .1, .03, .6])
sns.heatmap(TOTblockplot,ax=ax1, cmap = 'Spectral_r',vmin = 100, vmax = 1900,linewidths=.01,linecolor='black', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax1.spines[axis].set_visible(True)
    ax1.spines[axis].set_color('black')
    ax1.spines[axis].set_visible(True)
    ax1.spines[axis].set_color('black') 
ax2 = fig.add_axes([0.06, 0.18, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(TOTblockplot1,ax=ax2,cmap = 'Spectral_r',vmin = 100, vmax = 1900,linewidths=.01,linecolor='black', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax2.spines[axis].set_visible(True)
    ax2.spines[axis].set_color('black')
    ax2.spines[axis].set_visible(True)
    ax2.spines[axis].set_color('black')
ax3 = fig.add_axes([0.06, 0.35, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(TOTblockplot2,ax=ax3,  cmap = 'Spectral_r',vmin = 100, vmax = 1900,linewidths=.01,linecolor='black',yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax3.spines[axis].set_visible(True)
    ax3.spines[axis].set_color('black')
    ax3.spines[axis].set_visible(True)
    ax3.spines[axis].set_color('black') 
ax4 = fig.add_axes([0.06, 0.52, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(TOTblockplot3,ax=ax4, cmap = 'Spectral_r',vmin = 100, vmax = 1900,linewidths=.01,linecolor='black', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax4.spines[axis].set_visible(True)
    ax4.spines[axis].set_color('black')
    ax4.spines[axis].set_visible(True)
    ax4.spines[axis].set_color('black') 
ax5 = fig.add_axes([0.06, 0.69, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(TOTblockplot4,ax=ax5, cmap = 'Spectral_r',vmin =100, vmax = 1900,linewidths=.01,linecolor='black', yticklabels=False,xticklabels=False, cbar_ax = cbar_ax)
ax5.set_title("Time over Threshold (TOT)(ns)" )
for axis in ['top','bottom','left','right']:
    ax5.spines[axis].set_visible(True)
    ax5.spines[axis].set_color('black')
    ax5.spines[axis].set_visible(True)
    ax5.spines[axis].set_color('black') 
ax6 = fig.add_axes([0.46, 0.01, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(TOAblockplot,ax=ax6,vmin = 5, vmax = 160, cmap = 'Spectral_r',linewidths=.01,linecolor='black', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax6.spines[axis].set_visible(True)
    ax6.spines[axis].set_color('black')
    ax6.spines[axis].set_visible(True)
    ax6.spines[axis].set_color('black') 
ax7 = fig.add_axes([0.46, 0.18, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(TOAblockplot1,ax=ax7,vmin = 5, vmax = 160,cmap = 'Spectral_r',linewidths=.01,linecolor='black', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax7.spines[axis].set_visible(True)
    ax7.spines[axis].set_color('black')
    ax7.spines[axis].set_visible(True)
    ax7.spines[axis].set_color('black')
ax8 = fig.add_axes([0.46, 0.35, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(TOAblockplot2,ax=ax8,vmin = 5, vmax = 160,  cmap = 'Spectral_r',linewidths=.01,linecolor='black',yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax8.spines[axis].set_visible(True)
    ax8.spines[axis].set_color('black')
    ax8.spines[axis].set_visible(True)
    ax8.spines[axis].set_color('black') 
ax9 = fig.add_axes([0.46, 0.52, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(TOAblockplot3,ax=ax9,vmin = 5, vmax = 160, cmap = 'Spectral_r',linewidths=.01,linecolor='black', yticklabels=False,xticklabels=False, cbar = False)

for axis in ['top','bottom','left','right']:
    ax9.spines[axis].set_visible(True)
    ax9.spines[axis].set_color('black')
    ax9.spines[axis].set_visible(True)
    ax9.spines[axis].set_color('black') 
ax10 = fig.add_axes([0.46, 0.69, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(TOAblockplot4,ax=ax10,vmin = 5, vmax = 160, cmap = 'Spectral_r',linewidths=.01,linecolor='black', yticklabels=False,xticklabels=False, cbar_ax = cbar_ax2)
ax10.set_title("Time of Arrival from first hit (dTOA) (ns)" )
for axis in ['top','bottom','left','right']:
    ax10.spines[axis].set_visible(True)
    ax10.spines[axis].set_color('black')
    ax10.spines[axis].set_visible(True)
    ax10.spines[axis].set_color('black') 
plt.savefig('hitcomparision.eps', bbox_inches='tight')


In [ ]:
ax1 = fig.add_axes([0.06, 0.01, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(blockplot,ax=ax1, cmap = 'plasma', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax1.spines[axis].set_visible(True)
    ax1.spines[axis].set_color('black')
    ax1.spines[axis].set_visible(True)
    ax1.spines[axis].set_color('black') 
ax2 = fig.add_axes([0.28, 0.01, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(blockplot1,ax=ax2,cmap = 'plasma', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax2.spines[axis].set_visible(True)
    ax2.spines[axis].set_color('black')
    ax2.spines[axis].set_visible(True)
    ax2.spines[axis].set_color('black')
ax3 = fig.add_axes([0.5, 0.01, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(blockplot2,ax=ax3,  cmap = 'plasma',yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax3.spines[axis].set_visible(True)
    ax3.spines[axis].set_color('black')
    ax3.spines[axis].set_visible(True)
    ax3.spines[axis].set_color('black') 
ax4 = fig.add_axes([0.72, 0.01, 0.15, 0.15],
                   xticklabels=[], yticklabels = [])
sns.heatmap(blockplot3,ax=ax4, cmap = 'plasma', yticklabels=False,xticklabels=False, cbar = False)
for axis in ['top','bottom','left','right']:
    ax4.spines[axis].set_visible(True)
    ax4.spines[axis].set_color('black')
    ax4.spines[axis].set_visible(True)
    ax4.spines[axis].set_color('black') 

In [ ]:
def slicer(xdata,ydata,adata,tdata,del_t = 5000, frames = 25 ):
    #Takes in all the data, right now doesn't need ndata? Not sure how to incorporate
    # del_t is the slicesize in nanoseconds. 5000 is 5 μs & 1000000 is 1 ms
    gif_array = []
    int_array = []
    for i in range(frames):
        index = np.where((tdata >= 0 +(i*del_t)) & (tdata <= del_t+ (i*del_t)))
        xplot = xdata[index]
        yplot= ydata[index]
        tplot = tdata[index]
        aplot = adata[index]
        gif_fill = (xplot,yplot, tplot, aplot )
        gif_array.append(gif_fill)
        
    # creates the array of slices 
    for i in range(len(gif_array)):
        xfill,yfill,tfill, afill,  = gif_array[i]
        if i == 0:
            int_fill = (xfill,yfill, tfill, afill )
        if i > 0:
            xold, yold, told, aold = int_array[i-1]
            xadd = np.concatenate((xold,xfill))
            yadd = np.concatenate((yold,yfill))
            tadd = np.concatenate((told,tfill))
            aadd = np.concatenate((aold,afill))
            int_fill= (xadd,yadd,tadd,aadd)
        int_array.append(int_fill)
    # sums all the slices iteratively
    return(gif_array, int_array)

In [ ]:
def gif_plot(start, gif_array, int_array, del_t):
    for i in range(start,len(gif_array)):
        xplot1,yplot1,tplot1, aplot1,  = int_array[i]
        xplot2,yplot2,tplot2, aplot2,  = gif_array[i]    
        # creates an array the size of pixelboard to feed in data for seaborn heatmap
        TOT1 = np.zeros([256,256] )
        TOT2 = np.zeros([256,256] )
        TOT1.fill(np.nan)
        TOT2.fill( np.nan) 
        x1 = xplot1
        y1 = yplot1 
        z1 = aplot1
        x2 = xplot2
        y2 = yplot2
        z2 = aplot2
        min = np.str(i*(del_t/1000))
        max = np.str((del_t/1000)+ i*(del_t/1000))
        print(len(z2))
        for j in range(len(z2)):
            TOT2[np.int(x2[j]),np.int(y2[j])]=z2[j]
        for j in range(len(z1)):
            TOT1[np.int(x1[j]),np.int(y1[j])]=z1[j]
        # Plotting Functionality    
        fig, (ax1, ax2 ) = plt.subplots(ncols =2, figsize=(10, 6), sharex=True, sharey=True)
        plt.suptitle('Hits from '+min+' to '+max+' μs')
        cbar_ax = fig.add_axes([1,.2,.03,.6])
        sns.heatmap(TOT2,cmap='magma', ax = ax1,vmin = 200, vmax = 3000 ,square = True, cbar = False)
        sns.heatmap(TOT1,cmap='magma', ax = ax2, vmin = 200, vmax = 1000, square= True, cbar_ax = cbar_ax)
        fig.tight_layout() 
        i = np.str(i)
        plt.savefig('/home/andrew/Documents/BNL 2018/Coding/gifs/qe/fill_frame_'+i+'.png',  bbox_inches='tight')   
        # change to where you want to save 100  pngs
     

In [ ]:
def gif_creation(name, directory):
    # input name is a string
    # gif creation
    gif_name = name
    fps = .2      # frames per second
    os.chdir(directory)     # change this to directory you want 
    file_list = glob.glob('*.png') # Get all the pngs in the current directory   
    file_list = sorted( file_list, key=lambda f: int(''.join(filter(str.isdigit, f)))) 
    print(file_list)
    clip = mpy.ImageSequenceClip(file_list, fps=fps)
    clip.write_gif('{}.gif'.format(gif_name), fps=fps)   


In [ ]:
del_t = 10000
start = 0
gif_array, int_array = slicer(xdata,ydata,adata,tdata,del_t =del_t,  frames = 20)


In [ ]:
gif_plot(start, gif_array,int_array, del_t)